In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import keras
from glob import glob
import random

import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import load_img, img_to_array
from keras.utils.np_utils import to_categorical

In [2]:
imagePatches = glob('../input/breast-histopathology-images/IDC_regular_ps50_idx5/**/*.png', recursive=True)
for filename in imagePatches[0:10]:
    print(filename)

In [3]:
normal = [] # 0 = no cancer
cancer = [] # 1 = cancer

for filename in imagePatches:
    if filename.endswith("class0.png"):
         normal.append(filename)
    else:
        cancer.append(filename)

In [4]:
print(f"Length of Normal Class: {len(normal)}")
print(f"Length of Cancer Class: {len(cancer)}")

In [5]:
sampled_normal = random.sample(normal, 10000)
sampled_cancer = random.sample(cancer, 10000)

In [6]:
from matplotlib.image import imread
import cv2

def get_image_arrays(data, label):
    img_arrays = []
    for i in data:
        if i.endswith('.png'):
            img = cv2.imread(i ,cv2.IMREAD_COLOR)
            img_sized = cv2.resize(img, (50, 50), interpolation=cv2.INTER_LINEAR)
            img_arrays.append([img_sized, label])
    return img_arrays

In [7]:
normal_array = get_image_arrays(sampled_normal, 0)
cancer_array = get_image_arrays(sampled_cancer, 1)

In [8]:
normal_array[1]

In [9]:
test = cv2.imread('../input/breast-histopathology-images/IDC_regular_ps50_idx5/13689/1/13689_idx5_x801_y1501_class1.png' ,cv2.IMREAD_COLOR)
test.shape

In [10]:
def show_img(files):
    plt.figure(figsize= (10,10))
    ind = np.random.randint(0, len(files), 25)
    i=0
    for loc in ind:
        plt.subplot(5,5,i+1)
        sample = load_img(files[loc], target_size=(150,150))
        sample = img_to_array(sample)
        plt.axis("off")
        plt.imshow(sample.astype("uint8"))
        i+=1

In [11]:
show_img(imagePatches)

In [12]:
combined_data = np.concatenate((normal_array, cancer_array))
random.seed(42)
random.shuffle(combined_data)

In [13]:
X = []
y = []

for features,label in combined_data:
    X.append(features)
    y.append(label)

In [14]:
X = np.array(X).reshape(-1, 50, 50, 3)

In [15]:
X.shape

In [16]:
#Splitting data into train and test data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [17]:
img_size = 50
channel = 3

In [18]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_size,img_size, channel)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss ="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

In [19]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs= 5,
    verbose = 1)

In [20]:
history.params

In [21]:
#plotting the Accuracy of test and training sets
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#plotting the loss of test and training sets
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [22]:
model.save("H_breastcancer_CNN.h5")